# [HMM](https://zh.wikipedia.org/wiki/%E9%9A%90%E9%A9%AC%E5%B0%94%E5%8F%AF%E5%A4%AB%E6%A8%A1%E5%9E%8B)

假設我有三個骰子, 每次擲骰子時 **隨機** 抽一個骰子

![123](https://pic4.zhimg.com/435fb8d2d675dc0be95aedf27feb6b67_b.jpg)

第一次抽到 D6 擲出 1, 第二次抽到 D8 擲出 6, 第三次抽到 D8 擲出 3

![123](http://img.voidcn.com/vcimg/000/005/270/159_e20_46b.jpg)

我知道我有三个骰子，六面骰，四面骰，八面骰。我也知道我掷了十次的结果（1 6 3 5 2 7 3 5 2 4），我不知道每次用了那种骰子，我想知道最有可能的骰子序列。

**HMM 就是在解決 如果我今天只看的到 1 6 3, 如何知道 D6 D8 D8**

**HMM 就是在解決 如果我今天只看的到 1 6 3, 如何知道 D6 D8 D8**

**HMM 就是在解決 如果我今天只看的到 1 6 3, 如何知道 D6 D8 D8**

所以 HMM 的 **hidden** 就是指這裡的 D6 D8 D8, 又稱 ** 隱含層 **

- - -

假設只丟一個骰子, 擲出 1

![123](http://img.voidcn.com/vcimg/000/005/270/162_2e9_920.jpg)

則猜 D4, 因為 D4 出現 1 的 **機率** 最高, P(D4=1) = 1/4 > P(D6=1) = 1/6 > P(D8=1) = 1/8

假設丟兩個骰子, 擲出 1 6

![123](http://img.voidcn.com/vcimg/000/005/270/163_ef1_80c.jpg)

其實很簡單, 就是算 P(第一個骰子是D?, 第二個骰子是D?), 看看哪個 **機率** 最高

算兩個

### P(第一個骰子是D4, 第二個骰子是D4)

第一個骰子是 D4, 首先你必須先抽中 D4 (機率1/3), 抽中 D4 後要擲出 1(機率 1/4), 第二次要抽中 D4 (機率1/3), 抽中 D4 後要擲出 6(機率 0)

所以 P(第一個骰子是D4, 第二個骰子是D4) = 1/3 x 1/4 x 1/3 x 0 = 0

### P(第一個骰子是D4, 第二個骰子是D6)

第一個骰子是 D4, 首先你必須先抽中 D4 (機率1/3), 抽中 D4 後要擲出 1(機率 1/4), 第二次要抽中 D6 (機率1/3), 抽中 D6 後要擲出 6(機率 1/6)

所以 P(第一個骰子是D4, 第二個骰子是D6) = 1/3 x 1/4 x 1/3 x 1/6

同理, 如果有丟第三個骰子, 作法是一樣的

* [範例來源](https://www.zhihu.com/question/20962240)
* 求解是用 [Viterbi algorithm](https://en.wikipedia.org/wiki/Viterbi_algorithm), 可能在很多文獻裡會看到這個

- - -

## [工具](https://github.com/hmmlearn/hmmlearn)

以上述為範例

In [1]:
import numpy as np
from hmmlearn import hmm

In [9]:
model = hmm.MultinomialHMM(n_components=3) # 三個骰子
model.startprob_ = np.array([1/float(3), 1/float(3), 1/float(3)]) # 每次丟之前隨機抽
model.transmat_ = np.array([[1/float(3), 1/float(3), 1/float(3)], # 隱含層的轉移矩陣
                            [1/float(3), 1/float(3), 1/float(3)],
                            [1/float(3), 1/float(3), 1/float(3)]]) 
model.emissionprob_ = np.array([[1/float(4), 1/float(4), 1/float(4), 1/float(4), 0, 0, 0, 0], # 每個隱含層狀態的輸出機率
                                [1/float(6), 1/float(6), 1/float(6), 1/float(6), 1/float(6), 1/float(6), 0, 0],
                                [1/float(8), 1/float(8), 1/float(8), 1/float(8), 1/float(8), 1/float(8), 1/float(8), 1/float(8)]])

In [12]:
X = [[0], [5]] # 擲出 1 跟 6
model.predict(X) # D4 D6

array([0, 1])

- - -

## 應用

### 斷字

在一段文字中，我们可以将每个字按照他们在词中的位置进行标注，常用的标记有以下四个label：
* B，Begin，表示这个字是一个词的首字；
* M，Middle，表示这是一个词中间的字；
* E，End，表示这是一个词的尾字；
* S，Single，表示这是单字成词。

举例来说：“达观数据是企业大数据服务商”，经过模型后得到的理想标注序列是：“BMMESBEBMEBME”，最终还原的分词结果是“达观数据/是/企业/大数据/服务商”。

此時隱含層為 BMMESBEBMEBME, 就跟之前例子裡骰子種類一樣

不一樣在於

* 起始機率不再是 1/3, 因為斷字的情境裡 p(start = M) = p(start = E) = 0

* 隱含層的轉移矩陣 p(S -> B) = p(B -> M) = 1

* 每個隱含層狀態的輸出機率則更複雜, 比如 p(人|B) 表示在状态为B的情况下人字的概率

* [範例](http://www.leiphone.com/news/201608/IWvc75oJglAIsDvJ.html)

### 詞性標註

斷好詞之後, 达观数据 是 企业 大数据 服务商, 隱含層是 N V N N N, follow 一樣的概念

### 非文字應用

* [ANOMALY NETWORK INTRUSION DETECTION USING HIDDEN MARKOV MODEL](http://www.ijicic.org/ijicic-1508-0031.pdf)

* [維基](https://zh.wikipedia.org/wiki/%E9%9A%90%E9%A9%AC%E5%B0%94%E5%8F%AF%E5%A4%AB%E6%A8%A1%E5%9E%8B#.E9.9A.90.E9.A9.AC.E5.B0.94.E5.8F.AF.E5.A4.AB.E6.A8.A1.E5.9E.8B.E7.9A.84.E5.BA.94.E7.94.A8)